In [ ]:
from natsort import natsorted
from pathlib import Path
import sys, os

path2add = os.path.normpath(os.path.abspath(os.path.join(os.path.curdir, os.path.pardir)))
print(path2add)

if (not (path2add in sys.path)) :
    sys.path.append(path2add)

from beamtest_analysis_helper import toSingleDataFrame_newEventModel, DecodeBinary, plot_occupany_map, return_hist, plot_1d_TDC_histograms

In [ ]:
# ### In case, you're using "nem" files
# files = Path('/media/daq/X9/Run_nominal_alignment_12').glob('loop_*/*nem')
# print(files[0])
# print(files[-1])
# df = toSingleDataFrame_newEventModel(files=files)

In [ ]:
### In case, you're converting from binary files
input_path = "/media/daq/X9/Run_DESYFeb2024_Run_34/"
pattern = 'loop_*/TDC*.bin'
files = natsorted(list(Path(input_path).glob(pattern)))
print(files[0])
print(files[-1])
decode = DecodeBinary(firmware_key=0b0001, board_id=[0x17f0f, 0x17f0f, 0x17f0f, 0x17f0f], file_list=files)
df, _, _, _ = decode.decode_files() # hit dataframe, event dataframe, CRC dataframe, fillter dataframe
print(f'Nhits: {df.shape[0]}, Nevts: {df['evt'].nunique()}')

In [ ]:
chip_labels = [0, 1, 2, 3]
chip_names = ["ET2_BAR_4", "ET2p01_BAR_12", "ET2p01_BAR_13", "ET2_BB_1-3"]
offsets = [20, 20, 20, 20]
high_voltages = [250, 260, 210, 260]

chip_fignames = chip_names
chip_figtitles = [
    f"(Trigger) Bar 4 HV{high_voltages[0]}V OS:{offsets[0]}",
    f"Bar 12 HV{high_voltages[1]}V OS:{offsets[1]}",
    f"Bar 13 HV{high_voltages[2]}V OS:{offsets[2]}",
    f"BB 1-3 HV{high_voltages[3]}V OS:{offsets[3]}"]

In [ ]:
plot_occupany_map(df, board_ids=chip_labels, board_names=chip_names, tb_loc='cern')

In [ ]:
h_inclusive = return_hist(input_df=df, board_names=chip_names, board_ids=chip_labels, hist_bins=[100, 128, 128])
for iboard in chip_labels:
    plot_1d_TDC_histograms(h_inclusive, chip_names[iboard], 'cern', chip_figtitles[iboard], slide_friendly=True)
del h_inclusive